In [1]:
import ollama


In [3]:

res = ollama.chat(
	model="llava",
	messages=[
		{
			'role': 'user',
			'content': 'Describe this image:',
			'images': [r'C:\Users\swast\OneDrive\Desktop\runSwastik\Localinsight\documents\cat2.jpg']
		}
	]
)

print(res['message']['content'])

 This is a color photograph featuring a cat sitting upright on its hind legs. The cat has a dark fur coat and appears to be looking directly at the camera with an attentive expression. It's wearing a white collar that stands out against its fur. The cat's paws are positioned as if it's posing for the photo, with one paw resting on what looks like a window or table. The background is simple and nondescript, providing no additional context or setting for the photograph. There are no visible texts or distinctive markings that would provide more information about the scene or the subject. 


In [3]:
# ollama.create?x

In [4]:
# modelfile='''
# FROM tinyllama
# SYSTEM You are a helpful chat assistant that uses the given information to do what the user has asked them to do.
# '''

# model = ollama.create(model='assistant-tinyllama', modelfile=modelfile)

In [5]:
   # def display_response(self, response):
    #     for object in response.objects:
    #         if object.properties['media_type'] == "text":
    #             print(object.properties)
    #             print('-'*20)
    #         elif object.properties['media_type'] == "image":
    #             print(object.properties)
    #             display(Image(object.properties['path']))
    #             print('-'*20)


In [1]:
from database import DatabaseClient
from ollama import chat, generate

class ChatClient:

    def __init__(self, folder_path,  chat_model = 'llava', transform_model = 'phi3'):
        self.messages = []
        self.chat_model = chat_model
        self.transform_model = transform_model
        self.database = DatabaseClient(folder_path)
        
    
    def retriever(self, query = None, image_path = None):
        if image_path != None and query != None:
            return self.database.search_with_image(image_path) + self.database.search_with_text(query)  
        elif query == None:
            return self.database.search_with_image(image_path)
        elif image_path == None:
            return self.database.search_with_text(query)
        
    def transform_query(self, user_content):
        content = f"""Extract exactly 3-5 keywords from the following query, and return them in a comma-separated list with no additional text.
        Query: "{user_content}"
        Keywords: """
        return generate(self.transform_model, prompt=content, options={'temperature':0})['response']

    def handle_properties(self, properties):
        context = ""
        image_paths = []
        for property in properties:
            if property['media_type'] == 'text' and property['text'] not in context:
                context += "\n" + property['text']
            if property['media_type'] == 'image' and property['path'] not in image_paths:
                image_paths.append(property['path'])
        return context, image_paths

    def user_message(self, content, image_paths):
        return {
            'role' : 'user',
            'content' : content,
            'images' : image_paths
        }

    def input_text(self, user_content):
        transfomed_query = self.transform_query(user_content)
        context, image_paths = self.handle_properties(self.retriever(query=transfomed_query))
        return self.user_message(
            content=f"""Given context : {context}.
                        The user says :  {user_content}""", 
            image_paths=image_paths
            )

    def input_image(self, image_path):
        context, image_paths = self.handle_properties(self.retriever(image_path=image_path))
        if image_path not in image_paths:
            image_paths.append(image_path)
        return self.user_message(
            content=f"""Explain with help of images. 
                        Context : {context}.""", 
            image_paths=image_paths
            )
    
    def input_text_image(self, user_content, image_path):
        transfomed_query = self.transform_query(user_content)
        context, image_paths = self.handle_properties(self.retriever(query=transfomed_query, image_path=image_path))
        if image_path not in image_paths:
            image_paths.append(image_path)
        return self.user_message(
            content=f"""Given context : {context} and images.
                        User says : {user_content}.""",
            image_paths=image_paths
        )
    
    def interact(self):
        print("Type /exit to end.")
        while True:
            user_content = input("User Content: ")
            if user_content == "/exit":
                self.database.close_connection()
                break
            user_image_path = input("User Image Path: ")
            
            if user_content != '' and user_image_path != '':
                self.messages.append(self.input_text_image(
                    user_content=user_content,
                    image_path=user_image_path
                ))
            elif user_image_path == '':
                self.messages.append(self.input_text(user_content))
            elif user_content == '':
                self.messages.append(self.input_image(user_image_path))
            else:
                continue
            
            assistant_content = ''
            for chunk in chat(self.chat_model, stream=True, messages=self.messages, options={'temperature':0}):
                assistant_content += chunk['message']['content']
                print(chunk['message']['content'], end='', flush=True)
            
            print()
            print('-'*50)
            self.messages.append({
                    'role' : 'assistant',
                    'content' : assistant_content
                })




c:\Users\swast\AppData\Local\Programs\Python\Python312\Lib\site-packages\speech_recognition\__init__.py:7: DeprecationWarning: 'aifc' is deprecated and slated for removal in Python 3.13
  import aifc
c:\Users\swast\AppData\Local\Programs\Python\Python312\Lib\site-packages\speech_recognition\__init__.py:8: DeprecationWarning: 'audioop' is deprecated and slated for removal in Python 3.13
  import audioop


In [2]:
folder_path = r"C:\Users\swast\OneDrive\Desktop\runSwastik\Localinsight\documents"
chatbot = ChatClient(folder_path)

In [3]:
chatbot.interact()

Type /exit to end.
 Cats are domesticated mammals that belong to the Felidae family. They are known for their independent yet loving nature, as well as their graceful movements and expressive personalities. Despite their reputation for being aloof, cats can form strong bonds with their owners and be incredibly affectionate. Each cat has its own unique personality, ranging from playful and energetic to calm and reserved. Cats are carnivorous animals that primarily feed on meat, but they also require fresh water daily. They have a natural instinct for hunting and enjoy playing with toys that mimic prey. Cats are known for their agility and ability to jump high, as well as their keen senses of sight, sound, and smell. They are often considered low-maintenance pets because they are self-grooming and can be left alone for long periods of time. However, they do require regular playtime, exercise, and socialization to keep them happy and healthy. 
---------------------------------------------

# Testing

In [149]:
image_path = r"C:\Users\swast\OneDrive\Desktop\runSwastik\Localinsight\documents\dog1.jpg"
user_content = "Look Up :I wonder what Alice is upto. Then make a story with the information and images."
transformed_query = chatbot.transform_query(user_content)
transformed_query

'wonder,Alice,story,images,information'

In [145]:
chatbot.input_image(r"C:\Users\swast\OneDrive\Desktop\runSwastik\Localinsight\documents\dog1.jpg")

{'role': 'user',
 'content': 'Explain with help of images. \n                        Context : .',
 'images': None}

In [147]:
retrieved = chatbot.retriever(image_path=image_path)
retrieved

[{'text': None,
  'chunk_no': None,
  'image': '...',
  'path': 'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\dog1.jpg',
  'media_type': 'image'},
 {'text': None,
  'media_type': 'image',
  'image': '...',
  'path': 'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\dog1.jpg',
  'chunk_no': None},
 {'text': None,
  'chunk_no': None,
  'path': 'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\dog2.jpg',
  'image': '...',
  'media_type': 'image'},
 {'text': None,
  'chunk_no': None,
  'path': 'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\dog2.jpg',
  'image': '...',
  'media_type': 'image'},
 {'text': None,
  'chunk_no': None,
  'path': 'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\cat1.jpg',
  'image': '...',
  'media_type': 'image'}]

In [148]:
context, image_paths = chatbot.handle_properties(retrieved)
context, image_paths

('',
 ['C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\dog1.jpg',
  'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\dog2.jpg',
  'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\cat1.jpg'])

In [150]:
if image_path not in image_paths:
    image_paths.append(image_path)
user_message = chatbot.user_message(
        content=f"""Explain with help of images.
                    Do : {user_content}.""",
        image_paths=image_paths
    )
user_message

{'role': 'user',
 'content': 'Explain with help of images.\n                    Do : Look Up :I wonder what Alice is upto. Then make a story with the information and images..',
 'images': ['C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\dog1.jpg',
  'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\dog2.jpg',
  'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\cat1.jpg']}

In [156]:
chatbot.input_text_image(user_content="Look Up :I wonder what Alice is upto. Then make a story with the information and images.", image_path=r"C:\Users\swast\OneDrive\Desktop\runSwastik\Localinsight\documents\cat3.jpg")

{'role': 'user',
 'content': 'Given context : \nIn another moment down went Alice after it, never once considering how in the world she was to get out again.\nout-of-the-way things had happened lately, that Alice had begun to think that very few things indeed were really impossible.\nIt was all very well to say “Drink me,” but the wise little Alice was not going to do that in a hurry. “No, I’ll look first,” she said, “and see whether it’s marked ‘poison’ or not”; for she had read and images.\n                        User says : Look Up :I wonder what Alice is upto. Then make a story with the information and images..',
 'images': ['C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\cat3.jpg',
  'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\cat2.jpg',
  'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\cat1.jpg']}

In [155]:
chatbot.input_text(user_content="Look Up :I wonder what Alice is upto. Then make a story with the information and images.")#, image_path=r"C:\Users\swast\OneDrive\Desktop\runSwastik\Localinsight\documents\cat3.jpg")

{'role': 'user',
 'content': 'Given context : \nIn another moment down went Alice after it, never once considering how in the world she was to get out again.\nout-of-the-way things had happened lately, that Alice had begun to think that very few things indeed were really impossible.\nIt was all very well to say “Drink me,” but the wise little Alice was not going to do that in a hurry. “No, I’ll look first,” she said, “and see whether it’s marked ‘poison’ or not”; for she had read.\n                        The user says :  Look Up :I wonder what Alice is upto. Then make a story with the information and images.',
 'images': []}

In [154]:
chatbot.input_image(image_path=image_path)

{'role': 'user',
 'content': 'Explain with help of images. \n                        Context : .',
 'images': ['C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\dog1.jpg',
  'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\dog2.jpg',
  'C:\\Users\\swast\\OneDrive\\Desktop\\runSwastik\\Localinsight\\documents\\cat1.jpg']}